# Data Generators

In [1]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input



# Définir la taille cible des images pour ResNet50 (224x224)
target_size = (224, 224)

def resize_image(input_image_path, output_image_path, size=(224, 224)):
    """
    Redimensionne l'image à la taille spécifiée et la sauvegarde.

    Args:
        input_image_path (str): Chemin de l'image d'entrée.
        output_image_path (str): Chemin pour sauvegarder l'image redimensionnée.
        size (tuple, optionnel): Nouvelle taille de l'image. Par défaut (224, 224).
    """
    with Image.open(input_image_path) as img:
        resized_img = img.resize(size, Image.LANCZOS) # Méthodes de Redimensionnement LANCZOS
        resized_img.save(output_image_path)

# Fonction pour générer des images augmentées pour une seule classe
def generate_augmented_images(image_path, class_name, num_images=200, save_dir='images_data'):
    """
    Génère et sauvegarde des images augmentées pour une classe donnée.

    Args:
        image_path (str): Chemin vers l'image originale.
        class_name (str): Nom de la classe (par exemple, 'dog', 'cat').
        num_images (int, optionnel): Nombre d'images augmentées à générer. Par défaut, 100.
        save_dir (str, optionnel): Répertoire où sauvegarder les images augmentées. Par défaut, 'preview'.
    """
    # Charger et redimensionner l'image
    img = load_img(image_path, target_size=target_size)
    # Convertir en tableau NumPy
    x = img_to_array(img)
    # Ajouter une dimension de batch
    x = np.expand_dims(x, axis=0)
    # Prétraiter pour ResNet50
    x = preprocess_input(x)

    # Définir le générateur d'images augmentées
    datagen = ImageDataGenerator(
        rotation_range=40,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        brightness_range=(0.5, 1.5)
    )

    # Créer le répertoire de sauvegarde s'il n'existe pas
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    i = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=save_dir,
                              save_prefix=class_name,
                              save_format='jpeg'):
        i += 1
        if i >= num_images:
            break

# Générer des images augmentées pour les classes 'dog' et 'cat'
image_paths = {'dog': 'dog.jpg', 'cat': 'cat.jpg'}  # Dictionnaire associant les noms de classe aux chemins

# Répertoire pour les images redimensionnées
input_image_path = 'images'
resized_image_path = 'resized_images'
generated_image_path = 'Generated_images'

if not os.path.exists(resized_image_path):
    os.makedirs(resized_image_path)
if not os.path.exists(generated_image_path):
    os.makedirs(generated_image_path)

for class_name, image_file in image_paths.items():
    input_img_path = os.path.join(input_image_path, image_file)
    output_img_path = os.path.join(resized_image_path, f'{class_name}.jpg')
    resize_image(input_image_path=input_img_path, output_image_path=output_img_path)
    generate_augmented_images(image_path=output_img_path, class_name=class_name, save_dir=generated_image_path)



# Constants

In [2]:
# Fixed for our Cats & Dogs classes
NUM_CLASSES = 2

# Fixed for Cats & Dogs color images
#CHANNELS = 3

IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 100
BATCH_SIZE_VALIDATION = 100

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

TRAIN_DIR = 'Generated_images'

# Transfer Learning Network Model
**xyz_tf_kernels.h5** : Ce fichier de poids est utile pour la prédiction pure d'une image de test. Il utilise les poids pré-entraînés de ResNet50, ce qui signifie que la prédiction dépend entièrement des connaissances de ResNet50 et ne nécessite pas d'apprentissage supplémentaire de notre part.

**xyz_tf_kernels_NOTOP.h5** : Ce fichier de poids est utilisé pour le transfert d'apprentissage. Il exclut la couche supérieure (TOP) de ResNet50. Ces poids sont utilisés comme poids initiaux pour l'entraînement d'une nouvelle couche (ou plusieurs couches) sur de nouvelles images d'entraînement. En utilisant ces poids, on peut tirer parti des connaissances préalables de ResNet50 tout en adaptant le modèle à un nouvel ensemble de données ou à une tâche spécifique.

Ainsi, dans ce contexte, on utilise les poids xyz_tf_kernels_NOTOP.h5 pour ajuster le modèle ResNet50 à une nouvelle tâche, en conservant la majeure partie de l'apprentissage antérieur tout en permettant à une partie du modèle d'être spécifique à la nouvelle tâche.






In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Chargement des poids de ResNet50 à partir du fichier spécifique

resnet_weights_path = 'xyz_tf_kernels_NOTOP.h5'

# Création du modèle
model = Sequential()

# Ajout de la première couche à mon modèle : ajout ResNet50 au modèle
model.add(ResNet50(include_top=False, pooling=RESNET50_POOLING_AVERAGE, weights='imagenet'))

# Ajout de la seconde couche Dense pour la classification
model.add(Dense(NUM_CLASSES, activation=DENSE_LAYER_ACTIVATION))

# Désactivation de l'entraînement de la première couche (ResNet) car elle est déjà entraînée
#--> Cela implique de toutes les couches de ResNet : model.layers[0].trainable = False

# pour désactiver une couche à l interieur de ResNet50, par exemple la couche input
model.layers[0].layers[0].trainable = False
#pour désactvier la derbier couche de ResNet50
#model.layers[0].layers[len(model.layers[0].layers)-1].trainable = False

# Diviser les données en ensembles d'entraînement et de validation
train_dir = 'Generated_images/train'
test_dir = 'Generated_images/test'
train_images = [os.path.join(train_dir, img) for img in os.listdir(train_dir)]
test_images = [os.path.join(test_dir, img) for img in os.listdir(test_dir)]
train_images, val_images = train_test_split(train_images, test_size=0.2, random_state=42)

# Créer des générateurs d'images pour l'entraînement, la validation et le test
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'Generated_images/train',
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    'Generated_images/validation',
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'Generated_images/test',
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Entraînement du modèle avec validation
model.fit(train_generator, epochs=10, validation_data=val_generator)

# Évaluation du modèle sur les données de test
loss, accuracy = model.evaluate(test_generator)
print(f"Loss: {loss}, Accuracy: {accuracy}")
# Affichage des couches de ResNet50
for i, layer in enumerate(model.layers[0].layers):
    print(f"Layer {i}: {layer.name} - Trainable: {layer.trainable}")


94765736/94765736 [==============================] - 0s 0us/step
Layer 0: input_1 - Trainable: False
Layer 1: conv1_pad - Trainable: True
Layer 2: conv1_conv - Trainable: True
Layer 3: conv1_bn - Trainable: True
Layer 4: conv1_relu - Trainable: True
Layer 5: pool1_pad - Trainable: True
Layer 6: pool1_pool - Trainable: True
Layer 7: conv2_block1_1_conv - Trainable: True
Layer 8: conv2_block1_1_bn - Trainable: True
Layer 9: conv2_block1_1_relu - Trainable: True
Layer 10: conv2_block1_2_conv - Trainable: True
Layer 11: conv2_block1_2_bn - Trainable: True
Layer 12: conv2_block1_2_relu - Trainable: True
Layer 13: conv2_block1_0_conv - Trainable: True
Layer 14: conv2_block1_3_conv - Trainable: True
Layer 15: conv2_block1_0_bn - Trainable: True
Layer 16: conv2_block1_3_bn - Trainable: True
Layer 17: conv2_block1_add - Trainable: True
Layer 18: conv2_block1_out - Trainable: True
Layer 19: conv2_block2_1_conv - Trainable: True
Layer 20: conv2_block2_1_bn - Trainable: True
Layer 21: conv2_block2